# AutoGen အခြေခံ နမူနာ

ဒီကုဒ်နမူနာမှာ [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework ကို အသုံးပြုပြီး အခြေခံ Agent တစ်ခုကို ဖန်တီးမှာ ဖြစ်ပါတယ်။

ဒီနမူနာရဲ့ ရည်ရွယ်ချက်က နောက်ပိုင်းမှာ အခြား Agentic ပုံစံတွေကို အကောင်အထည်ဖော်တဲ့အခါ အသုံးပြုမယ့် လုပ်ဆောင်မှုအဆင့်တွေကို ပြသဖို့ ဖြစ်ပါတယ်။


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Client ကို ဖန်တီးပါ

ဤနမူနာတွင် [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ကို အသုံးပြု၍ LLM ကို ဝင်ရောက်အသုံးပြုမည်ဖြစ်သည်။

`model` ကို `gpt-4o-mini` အဖြစ် သတ်မှတ်ထားသည်။ GitHub Models marketplace တွင် ရရှိနိုင်သော အခြား model တစ်ခုကို ပြောင်းလဲကြည့်ပြီး ရလဒ်များကို ကွဲပြားမှုကို ကြည့်ရှုပါ။

အမြန်စမ်းသပ်မှုအနေနှင့် `What is the capital of France` ဆိုသော ရိုးရှင်းသော prompt ကို run လုပ်ကြည့်မည်။


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## အေးဂျင့်ကို သတ်မှတ်ခြင်း

`client` ကို စနစ်တကျ ပြင်ဆင်ပြီး အလုပ်လုပ်နေသည်ကို အတည်ပြုပြီးနောက် `AssistantAgent` တစ်ခုကို ဖန်တီးပါမည်။ အေးဂျင့်တစ်ခုစီကို အောက်ပါအတိုင်း သတ်မှတ်နိုင်ပါသည် -  
**name** - အေးဂျင့်ကို ရည်ညွှန်းရန် အကျဉ်းချုပ်အမည်တစ်ခု။  
**model_client** - ယခင်အဆင့်တွင် ဖန်တီးထားသော client။  
**tools** - အေးဂျင့်အနေဖြင့် တာဝန်ကို ပြည့်စုံစွာ ဆောင်ရွက်ရန် အသုံးပြုနိုင်သော tools များ။  
**system_message** - LLM ၏ တာဝန်၊ အပြုအမူနှင့် စကားသံကို သတ်မှတ်သော metaprompt။

သင်၏ system message ကို ပြောင်းလဲပြီး LLM ၏ တုံ့ပြန်မှုကို ကြည့်ရှုနိုင်ပါသည်။ `tools` ကို Lesson #4 တွင် ဆွေးနွေးမည်။


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## အေးဂျင့်ကို အလုပ်လုပ်စေပါ 

အောက်ပါ function သည် အေးဂျင့်ကို အလုပ်လုပ်စေမည်ဖြစ်သည်။ `on_message` method ကို အသုံးပြု၍ အသစ်ရရှိသော စာသားနှင့်အတူ အေးဂျင့်၏ အခြေအနေကို အပ်ဒိတ်လုပ်ပါသည်။

ဤအခါတွင်၊ အသုံးပြုသူထံမှ `"Plan me a great sunny vacation"` ဟုဆိုသော အသစ်သော စာသားဖြင့် အခြေအနေကို အပ်ဒိတ်လုပ်ပါသည်။

သင်သည် စာသားအကြောင်းအရာကို ပြောင်းလဲပြီး LLM က ဘယ်လို ကွဲပြားစွာ တုံ့ပြန်မည်ကို ကြည့်ရှုနိုင်ပါသည်။


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူရင်းဘာသာစကားဖြင့် အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွဲအချော်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
